# [How to Do Deep Learning on Graphs with Graph Convolutional Networks](https://github.com/TobiasSkovgaardJepsen/posts/blob/master/HowToDoDeepLearningOnGraphsWithGraphConvolutionalNetworks/Part2_SemiSupervisedLearningWithSpectralGraphConvolutions/notebook.ipynb)
## Part 2: Semi-Supervised Learning with Spectral Graph Convolutions
This notebook accompanies my Medium article with the above title for readers to try out and explore graph convolutional networks for themselves. You can find the article [here](https://towardsdatascience.com/how-to-do-deep-learning-on-graphs-with-graph-convolutional-networks-62acf5b143d0). To run the notebook, install the packages specified in the accompanying ```requirements.txt``` file.

#  Load Karate Club

In [1]:
from gcnclass import SpectralRule, LogisticRegressor, load_karate_club

In [2]:
from networkx import to_numpy_matrix, degree_centrality, betweenness_centrality, shortest_path_length
import mxnet.ndarray as nd

zkc = load_karate_club()

A = to_numpy_matrix(zkc.network)
A = nd.array(A)

X_train = zkc.X_train.flatten()
y_train = zkc.y_train
X_test = zkc.X_test.flatten()
y_test = zkc.y_test

In [3]:
X_train


[[ 0.]
 [33.]]
<NDArray 2x1 @cpu(0)>

# Layer Implementations

In [4]:
from mxnet.gluon import HybridBlock
from mxnet.gluon.nn import Activation
import mxnet.ndarray as nd

In [5]:
class LogisticRegressor(HybridBlock):
    def __init__(self, in_units, **kwargs):
        super().__init__(**kwargs)
        with self.name_scope():
            self.w = self.params.get(
                'w', shape=(1, in_units)
            )

            self.b = self.params.get(
                'b', shape=(1, 1)
            )

    def hybrid_forward(self, F, X, w, b):
        # Change shape of b to comply with MXnet addition API
        b = F.broadcast_axis(b, axis=(0,1), size=(34, 1))
        y = F.dot(X, w, transpose_b=True) + b

        return F.sigmoid(y)

# Models

In [6]:
from mxnet.gluon.nn import HybridSequential, Activation
from mxnet.ndarray import array
from mxnet.initializer import One, Uniform, Xavier
from mxnet.gluon.loss import SigmoidBinaryCrossEntropyLoss

def build_features(A, X):
    hidden_layer_specs = [(4, 'tanh'), (2, 'tanh'),] # Format: (units in layer, activation function)
    in_units = in_units=X.shape[1]
  
    features = HybridSequential()
    with features.name_scope():
        for i, (layer_size, activation_func) in enumerate(hidden_layer_specs):
            layer = SpectralRule(
                A, in_units=in_units, out_units=layer_size, 
                activation=activation_func)
            features.add(layer)

            in_units = layer_size
    return features, in_units

def build_model(A, X):
    model = HybridSequential()
    hidden_layer_specs = [(4, 'tanh'), (2, 'tanh')]
    in_units = in_units=X.shape[1]

    with model.name_scope():
        features, out_units = build_features(A, X)
        model.add(features)

        classifier = LogisticRegressor(out_units)
        model.add(classifier)

    model.hybridize()
    model.initialize(Uniform(1))

    return model, features

## Model 1: Identity Matrix as Features

In [7]:
X_1 = I = nd.eye(*A.shape)
model_1, features_1 = build_model(A, X_1)

In [8]:
model_1(X_1)


[[0.5106775 ]
 [0.5095114 ]
 [0.51096225]
 [0.51542187]
 [0.523025  ]
 [0.512526  ]
 [0.511652  ]
 [0.50208426]
 [0.5050116 ]
 [0.51883554]
 [0.5406651 ]
 [0.49158806]
 [0.5110878 ]
 [0.4936631 ]
 [0.5026252 ]
 [0.48504698]
 [0.5104648 ]
 [0.51193994]
 [0.5220448 ]
 [0.50951165]
 [0.50140274]
 [0.5098576 ]
 [0.53033787]
 [0.51043373]
 [0.50124794]
 [0.52045095]
 [0.50941336]
 [0.49507195]
 [0.5021959 ]
 [0.5068235 ]
 [0.5085803 ]
 [0.51637703]
 [0.50986254]
 [0.5153003 ]]
<NDArray 34x1 @cpu(0)>

## Model 2: Distance to Administrator and Instructor as Additional Features

In [9]:
X_2 = nd.zeros((A.shape[0], 2))
node_distance_instructor = shortest_path_length(zkc.network, target=33)
node_distance_administrator = shortest_path_length(zkc.network, target=0)

for node in zkc.network.nodes():
    X_2[node][0] = node_distance_administrator[node]
    X_2[node][1] = node_distance_instructor[node]

In [10]:
X_2 = nd.concat(X_1, X_2)
model_2, features_2 = build_model(A, X_2)
model_2(X_2)


[[0.456614  ]
 [0.45822805]
 [0.456899  ]
 [0.457218  ]
 [0.45603466]
 [0.4640951 ]
 [0.4631094 ]
 [0.45584652]
 [0.4608328 ]
 [0.47084248]
 [0.4593098 ]
 [0.4513544 ]
 [0.46219462]
 [0.47803804]
 [0.46955827]
 [0.49520862]
 [0.46266046]
 [0.4590963 ]
 [0.49145314]
 [0.46135917]
 [0.48991072]
 [0.4575545 ]
 [0.48936   ]
 [0.4587967 ]
 [0.46603325]
 [0.47440898]
 [0.48096892]
 [0.47632974]
 [0.47681022]
 [0.47006816]
 [0.47394764]
 [0.46878797]
 [0.4676332 ]
 [0.49637428]]
<NDArray 34x1 @cpu(0)>

# Train and Test Models

In [11]:
%time
from mxnet import autograd
from mxnet.gluon import Trainer
from mxnet.ndarray import sum as ndsum
import numpy as np

def train(model, features, X, X_train, y_train, epochs):
    cross_entropy = SigmoidBinaryCrossEntropyLoss(from_sigmoid=True)
    trainer = Trainer(model.collect_params(), 'sgd', {'learning_rate': 0.001, 'momentum': 1})

    feature_representations = [features(X).asnumpy()]

    for e in range(1, epochs + 1):
        cum_loss = 0
        cum_preds = []

        for i, x in enumerate(X_train):
            y = array(y_train)[i]
            with autograd.record():
                preds = model(X)[x]
                loss = cross_entropy(preds, y)
            loss.backward()
            trainer.step(1)

            cum_loss += loss[0].asscalar()
            cum_preds += [preds[0].asscalar()]

        feature_representations.append(features(X).asnumpy())
            
        if (e % (epochs//10)) == 0:
            print(f"Epoch {e}/{epochs} -- Loss: {cum_loss: .4f}")
            print(cum_preds)
    return feature_representations

def predict(model, X, nodes):
    preds = model(X)[nodes].asnumpy().flatten()
    return  np.where(preds >= 0.5, 1, 0)

CPU times: user 2 µs, sys: 1 µs, total: 3 µs
Wall time: 4.77 µs


## Performance of Model 1

In [12]:
features_1

HybridSequential(
  (0): SpectralRule(
    (activation): Activation(tanh)
  )
  (1): SpectralRule(
    (activation): Activation(tanh)
  )
)

In [13]:
X_train


[[ 0.]
 [33.]]
<NDArray 2x1 @cpu(0)>

In [14]:
X_1


[[1. 0. 0. ... 0. 0. 0.]
 [0. 1. 0. ... 0. 0. 0.]
 [0. 0. 1. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 1. 0. 0.]
 [0. 0. 0. ... 0. 1. 0.]
 [0. 0. 0. ... 0. 0. 1.]]
<NDArray 34x34 @cpu(0)>

In [15]:
from sklearn.metrics import classification_report

feature_representations_1 = train(model_1, features_1, X_1, X_train, y_train, epochs=1000)
y_pred_1 = predict(model_1, X_1, X_test)


Epoch 100/1000 -- Loss:  1.3263
[0.49567983, 0.4644357]
Epoch 200/1000 -- Loss:  0.7557
[0.6250237, 0.24851143]
Epoch 300/1000 -- Loss:  0.0460
[0.9604993, 0.005714733]
Epoch 400/1000 -- Loss:  0.0012
[0.9992391, 0.00042324717]
Epoch 500/1000 -- Loss:  0.0001
[0.9999902, 4.623498e-05]
Epoch 600/1000 -- Loss:  0.0000
[0.9999999, 5.2589753e-06]
Epoch 700/1000 -- Loss:  0.0000
[1.0, 6.0081516e-07]
Epoch 800/1000 -- Loss:  0.0000
[1.0, 6.86608e-08]
Epoch 900/1000 -- Loss:  0.0000
[1.0, 7.847022e-09]
Epoch 1000/1000 -- Loss:  0.0000
[1.0, 8.968142e-10]


In [16]:
print(classification_report(y_test.asnumpy(), y_pred_1))

              precision    recall  f1-score   support

         0.0       0.75      0.38      0.50        16
         1.0       0.58      0.88      0.70        16

    accuracy                           0.62        32
   macro avg       0.67      0.62      0.60        32
weighted avg       0.67      0.62      0.60        32



## Performance of Model 2

In [17]:
feature_representations_2= train(model_2, features_2, X_2, X_train, y_train, epochs=250)
y_pred_2 = predict(model_2, X_2, X_test)
print(classification_report(y_test.asnumpy(), y_pred_2))

Epoch 25/250 -- Loss:  1.4502
[0.46630818, 0.49708325]
Epoch 50/250 -- Loss:  1.4058
[0.48751295, 0.49711376]
Epoch 75/250 -- Loss:  1.3709
[0.5132726, 0.5053706]
Epoch 100/250 -- Loss:  1.3566
[0.5332241, 0.5170041]
Epoch 125/250 -- Loss:  1.3161
[0.53867733, 0.5021284]
Epoch 150/250 -- Loss:  1.1604
[0.5339666, 0.41315266]
Epoch 175/250 -- Loss:  0.8730
[0.5482581, 0.23811252]
Epoch 200/250 -- Loss:  0.5649
[0.625712, 0.09159252]
Epoch 225/250 -- Loss:  0.3007
[0.7656582, 0.03307582]
Epoch 250/250 -- Loss:  0.1270
[0.8933893, 0.014171541]
              precision    recall  f1-score   support

         0.0       0.83      0.94      0.88        16
         1.0       0.93      0.81      0.87        16

    accuracy                           0.88        32
   macro avg       0.88      0.88      0.87        32
weighted avg       0.88      0.88      0.87        32

